# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC


warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
### costanti
# clean captures
clean_capture = 'captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

# ping flooding captures 1 min
pf_capture_1 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS5m-0,5h_1.pcap'
pf_capture_2 = 'captures\\captures1_v2\\captures2\\captures2\\pingFloodDDoS\\eth2dump-pingFloodDDoS5m-0.5h_1.json'
pf_capture_3 = 'captures\\captures3\\pingFloodDDoS\\eth2dump-pingFloodDDoS5m+0.5h_1.json'

levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']

### variabili

In [3]:
### estrazione dati
# packets clean capture
packets_cc = fn.load_packets_from_file(clean_capture)

# packets attack captures (es. packets ping flooding attack 1m per cattura di 0.5 = pkt_pf_1m_05h)
pkt_pf_1m_05h = fn.load_packets_from_file(pf_capture_1)
pkt_pf_5m_05h = fn.load_packets_from_file(pf_capture_2)
pkt_pf_15m_05h = fn.load_packets_from_file(pf_capture_3)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 264: character maps to <undefined>

In [4]:
### features
# clean features
features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)

# ping flooding attack features
features_pf_1m_05h = fn.extract_features_from_packets(pkt_pf_1m_05h, levels_of_interest)
features_pf_5m_05h = fn.extract_features_from_packets(pkt_pf_5m_05h, levels_of_interest)
features_pf_15m_05h =fn.extract_features_from_packets(pkt_pf_15m_05h, levels_of_interest)

In [5]:
### Dataframes
# df clean capture
df_clean = pd.DataFrame(features_cc)

# df ping flooding attacks captures
df_pf_1m_05h = pd.DataFrame(features_pf_1m_05h)
df_pf_5m_05h = pd.DataFrame(features_pf_5m_05h)
df_pf_15m_05h = pd.DataFrame(features_pf_15m_05h)



df_pf_1m_05h['label']=1
df_pf_5m_05h['label']=1
df_pf_15m_05h['label']=1
df_clean['label']=0
pd_concatenation = pd.concat([df_clean,df_pf_1m_05h, df_pf_5m_05h, df_pf_15m_05h])

In [6]:
df_colonne_senza_nan = pd_concatenation.loc[:, pd_concatenation.isna().sum() == 0].copy()


In [7]:
len(df_colonne_senza_nan.select_dtypes(include=['object']).columns)

60

In [8]:
print(df_colonne_senza_nan['label'])

0        0
1        0
2        0
3        0
4        0
        ..
29687    1
29688    1
29689    1
29690    1
29691    1
Name: label, Length: 196385, dtype: int64


In [9]:
df_colonne_senza_nan.select_dtypes(include=['object']).iloc[:10,:10]

,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len
0,1,"Sep 8, 2018 22:54:30.055515000 ora legale Eur...","Sep 8, 2018 20:54:30.055515000 UTC",1536440070.055515000,0.000000000,0.000000000,0.000000000,0.000000000,1,64
1,1,"Sep 8, 2018 22:54:30.111272000 ora legale Eur...","Sep 8, 2018 20:54:30.111272000 UTC",1536440070.111272000,0.000000000,0.055757000,0.055757000,0.055757000,2,60
2,1,"Sep 8, 2018 22:54:30.205937000 ora legale Eur...","Sep 8, 2018 20:54:30.205937000 UTC",1536440070.205937000,0.000000000,0.094665000,0.094665000,0.150422000,3,66
3,1,"Sep 8, 2018 22:54:30.209039000 ora legale Eur...","Sep 8, 2018 20:54:30.209039000 UTC",1536440070.209039000,0.000000000,0.003102000,0.003102000,0.153524000,4,85
4,1,"Sep 8, 2018 22:54:30.423275000 ora legale Eur...","Sep 8, 2018 20:54:30.423275000 UTC",1536440070.423275000,0.000000000,0.214236000,0.214236000,0.367760000,5,60
5,1,"Sep 8, 2018 22:54:30.517529000 ora legale Eur...","Sep 8, 2018 20:54:30.517529000 UTC",1536440070.517529000,0.000000000,0.094254000,0.094254000,0.462014000,6,66
6,1,"Sep 8, 2018 22:54:30.528387000 ora legale Eur...","Sep 8, 2018 20:54:30.528387000 UTC",1536440070.528387000,0.000000000,0.010858000,0.010858000,0.472872000,7,85
7,1,"Sep 8, 2018 22:54:30.735268000 ora legale Eur...","Sep 8, 2018 20:54:30.735268000 UTC",1536440070.735268000,0.000000000,0.206881000,0.206881000,0.679753000,8,60
8,1,"Sep 8, 2018 22:54:30.751871000 ora legale Eur...","Sep 8, 2018 20:54:30.751871000 UTC",1536440070.751871000,0.000000000,0.016603000,0.016603000,0.696356000,9,82
9,1,"Sep 8, 2018 22:54:30.829737000 ora legale Eur...","Sep 8, 2018 20:54:30.829737000 UTC",1536440070.829737000,0.000000000,0.077866000,0.077866000,0.774222000,10,66


### Feature tipo intero
frame.frame.encap_type
frame.frame.number
frame.frame.len
eth.eth.dst_tree.eth.dst.oui	
frame.frame.cap_len	frame.frame.marked	frame.frame.ignored
eth.eth.dst_tree.eth.dst.lg_raw[0]	
eth.eth.dst_tree.eth.addr.oui
eth.eth.dst_tree.eth.dst.lg
eth.eth.dst_tree.eth.lg_raw[0]	eth.eth.dst_tree.eth.lg	eth.eth.dst_tree.eth.dst.ig_raw[0]	eth.eth.dst_tree.eth.dst.ig	eth.eth.dst_tree.eth.ig_raw[0]	eth.eth.dst_tree.eth.ig
eth.eth.src
eth.eth.src_tree.eth.src.oui	
eth.eth.src_tree.eth.addr.oui
eth.eth.src_tree.eth.src.lg_raw[0]	eth.eth.src_tree.eth.src.lg	eth.eth.src_tree.eth.lg_raw[0]	eth.eth.src_tree.eth.lg	eth.eth.src_tree.eth.src.ig_raw[0]	eth.eth.src_tree.eth.src.ig	eth.eth.src_tree.eth.ig_raw[0]	eth.eth.src_tree.eth.ig
#### ESADECIMALE
eth.eth.dst_raw[0]	 eth.eth.dst_tree.eth.dst_resolved_raw[0] eth.eth.dst_tree.eth.dst.oui_raw[0] eth.eth.dst_tree.eth.addr_raw[0] eth.eth.dst_tree.eth.addr.oui_raw[0]
eth.eth.src_raw[0]
eth.eth.src_tree.eth.src_resolved_raw[0]	
eth.eth.src_tree.eth.src.oui_raw[0]
eth.eth.src_tree.eth.src.oui_resolved_raw[0]
eth.eth.src_tree.eth.addr_raw[0]
eth.eth.src_tree.eth.addr
eth.eth.src_tree.eth.addr_resolved_raw[0]
eth.eth.src_tree.eth.addr.oui_raw[0]
eth.eth.src_tree.eth.addr.oui_resolved_raw[0]				
#### MAC ADDRESS	
eth.eth.dst mac address eth.eth.dst_tree.eth.addr eth.eth.dst_tree.eth.addr_resolved_raw[0] 	 (da rimuovere : e conversione in esadecimale)

### Feature tipo float
frame.frame.time_epoch
frame.frame.offset_shift
frame.frame.time_delta
frame.frame.time_delta_displayed
frame.frame.time_relative
### Feature tipo string
frame.frame.protocols	da ricavare i protocolli contenuti all'interno del frame
#### DA SCARTARE
'frame.frame.coloring_rule.name'
'frame.frame.coloring_rule.string'
'eth.eth.dst_tree.eth.dst_resolved'
'eth.eth.dst_tree.eth.addr_resolved'
'eth.eth.src_tree.eth.src_resolved'
'eth.eth.src_tree.eth.src.oui_resolved'	
'eth.eth.src_tree.eth.addr_resolved'
'eth.eth.src_tree.eth.addr.oui_resolved'

In [10]:
pd_nona_bck = df_colonne_senza_nan.copy()
df_colonne_senza_nan.drop(columns=['frame.frame.coloring_rule.name','frame.frame.coloring_rule.string', 'eth.eth.dst_tree.eth.dst_resolved', 'eth.eth.dst_tree.eth.addr_resolved', 'eth.eth.src_tree.eth.src_resolved', 'eth.eth.src_tree.eth.src.oui_resolved', 'eth.eth.src_tree.eth.addr_resolved', 'eth.eth.src_tree.eth.addr.oui_resolved'], inplace=True)
column_names = df_colonne_senza_nan.columns.tolist()
print(column_names)


['frame.frame.encap_type', 'frame.frame.time', 'frame.frame.time_utc', 'frame.frame.time_epoch', 'frame.frame.offset_shift', 'frame.frame.time_delta', 'frame.frame.time_delta_displayed', 'frame.frame.time_relative', 'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len', 'frame.frame.marked', 'frame.frame.ignored', 'frame.frame.protocols', 'eth.eth.dst_raw[0]', 'eth.eth.dst_raw[1]', 'eth.eth.dst_raw[2]', 'eth.eth.dst_raw[3]', 'eth.eth.dst_raw[4]', 'eth.eth.dst', 'eth.eth.dst_tree.eth.dst_resolved_raw[0]', 'eth.eth.dst_tree.eth.dst_resolved_raw[1]', 'eth.eth.dst_tree.eth.dst_resolved_raw[2]', 'eth.eth.dst_tree.eth.dst_resolved_raw[3]', 'eth.eth.dst_tree.eth.dst_resolved_raw[4]', 'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui_raw[1]', 'eth.eth.dst_tree.eth.dst.oui_raw[2]', 'eth.eth.dst_tree.eth.dst.oui_raw[3]', 'eth.eth.dst_tree.eth.dst.oui_raw[4]', 'eth.eth.dst_tree.eth.dst.oui', 'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr_raw[1]', 'eth.e

In [11]:
len(df_colonne_senza_nan)

196385

In [12]:
fn.df_convert(df_colonne_senza_nan)
df_colonne_senza_nan

,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,...,eth.eth.src_tree.eth.src.ig_raw[3],eth.eth.src_tree.eth.src.ig_raw[4],eth.eth.src_tree.eth.src.ig,eth.eth.src_tree.eth.ig_raw[0],eth.eth.src_tree.eth.ig_raw[1],eth.eth.src_tree.eth.ig_raw[2],eth.eth.src_tree.eth.ig_raw[3],eth.eth.src_tree.eth.ig_raw[4],eth.eth.src_tree.eth.ig,label
0,1,"Sep 8, 2018 22:54:30.055515000 ora legale Eur...","Sep 8, 2018 20:54:30.055515000 UTC",1.536440e+09,0.0,0.000000,0.000000,0.000000,1,64,...,65536,2,0,0,6,3,65536,2,0,0
1,1,"Sep 8, 2018 22:54:30.111272000 ora legale Eur...","Sep 8, 2018 20:54:30.111272000 UTC",1.536440e+09,0.0,0.055757,0.055757,0.055757,2,60,...,65536,2,0,0,6,3,65536,2,0,0
2,1,"Sep 8, 2018 22:54:30.205937000 ora legale Eur...","Sep 8, 2018 20:54:30.205937000 UTC",1.536440e+09,0.0,0.094665,0.094665,0.150422,3,66,...,65536,2,0,0,6,3,65536,2,0,0
3,1,"Sep 8, 2018 22:54:30.209039000 ora legale Eur...","Sep 8, 2018 20:54:30.209039000 UTC",1.536440e+09,0.0,0.003102,0.003102,0.153524,4,85,...,65536,2,0,0,6,3,65536,2,0,0
4,1,"Sep 8, 2018 22:54:30.423275000 ora legale Eur...","Sep 8, 2018 20:54:30.423275000 UTC",1.536440e+09,0.0,0.214236,0.214236,0.367760,5,60,...,65536,2,0,0,6,3,65536,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29687,1,"May 21, 2018 12:59:40.703070000 ora legale Eur...","May 21, 2018 10:59:40.703070000 UTC",1.526900e+09,0.0,0.009229,0.009229,1798.146323,29688,60,...,65536,2,0,0,6,3,65536,2,0,1
29688,1,"May 21, 2018 12:59:40.711341000 ora legale Eur...","May 21, 2018 10:59:40.711341000 UTC",1.526900e+09,0.0,0.008271,0.008271,1798.154594,29689,60,...,65536,2,0,0,6,3,65536,2,0,1
29689,1,"May 21, 2018 12:59:40.712171000 ora legale Eur...","May 21, 2018 10:59:40.712171000 UTC",1.526900e+09,0.0,0.000830,0.000830,1798.155424,29690,66,...,65536,2,0,0,6,3,65536,2,0,1
29690,1,"May 21, 2018 12:59:40.770448000 ora legale Eur...","May 21, 2018 10:59:40.770448000 UTC",1.526900e+09,0.0,0.058277,0.058277,1798.213701,29691,64,...,65536,2,0,0,6,3,65536,2,0,1


In [13]:
print(df_colonne_senza_nan.dtypes)

frame.frame.encap_type              int32
frame.frame.time                   object
frame.frame.time_utc               object
frame.frame.time_epoch            float64
frame.frame.offset_shift          float64
                                   ...   
eth.eth.src_tree.eth.ig_raw[2]      int64
eth.eth.src_tree.eth.ig_raw[3]      int64
eth.eth.src_tree.eth.ig_raw[4]      int64
eth.eth.src_tree.eth.ig             int32
label                               int64
Length: 141, dtype: object


In [14]:
df_colonne_senza_nan.drop(columns=['frame.frame.time_utc','frame.frame.time','frame.frame.protocols'],inplace=True)


In [15]:

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Supponiamo che 'label' sia la colonna target e tutte le altre colonne siano features
X = df_colonne_senza_nan.drop(columns=['label'])
y = df_colonne_senza_nan['label']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



In [16]:
# Definire il numero di feature da selezionare
k = 68

# Applicare SelectKBest con la funzione di score f_classif (ANOVA)
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X_scaled, y)

# Ottenere i punteggi delle features
scores = selector.scores_

# Ottenere le feature selezionate
selected_features = selector.get_support(indices=True)

# Ottenere i nomi delle colonne selezionate
selected_feature_names = X.columns[selected_features].tolist()

print("Le 68 features migliori selezionate dall'ANOVA sono:")
print(selected_feature_names)


# Se desideri memorizzarle in una variabile lista
features_list = selected_feature_names
print(features_list)



Le 68 features migliori selezionate dall'ANOVA sono:
['frame.frame.time_epoch', 'frame.frame.time_delta', 'frame.frame.time_delta_displayed', 'frame.frame.time_relative', 'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len', 'eth.eth.dst_raw[0]', 'eth.eth.dst', 'eth.eth.dst_tree.eth.dst_resolved_raw[0]', 'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui', 'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr', 'eth.eth.dst_tree.eth.addr_resolved_raw[0]', 'eth.eth.dst_tree.eth.addr.oui_raw[0]', 'eth.eth.dst_tree.eth.addr.oui', 'eth.eth.dst_tree.eth.dst.lg_raw[0]', 'eth.eth.dst_tree.eth.dst.lg', 'eth.eth.dst_tree.eth.lg_raw[0]', 'eth.eth.dst_tree.eth.lg', 'eth.eth.dst_tree.eth.dst.ig_raw[0]', 'eth.eth.dst_tree.eth.dst.ig', 'eth.eth.dst_tree.eth.ig_raw[0]', 'eth.eth.dst_tree.eth.ig', 'eth.eth.src_raw[0]', 'eth.eth.src', 'eth.eth.src_tree.eth.src_resolved_raw[0]', 'eth.eth.src_tree.eth.src.oui_raw[0]', 'eth.eth.src_tree.eth.src.oui', 'eth.eth.src_tree.

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  0   2   9  10  12  13  14  15  18  19  20  21  23  24  25  26  29  30
  31  32  35  36  37  38  40  41  42  43  46  47  48  49  52  53  54  55
  58  59  60  61  64  65  66  67  70  71  72  73  76  77  78  79  81  82
  83  84  87  88  89  90  92  93  94  95  98  99 100 101 103 104 105 106
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [17]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

# Supponiamo che 'df' sia il tuo DataFrame e 'label' sia la colonna target
X = df_colonne_senza_nan.drop(columns=['label'])
y = df_colonne_senza_nan['label']

# Identifica le colonne costanti
constant_features = [col for col in X.columns if X[col].nunique() == 1]
print("Colonne costanti:")
print(constant_features)

# Rimuovi le colonne costanti
X = X.drop(columns=constant_features)

# Normalizza le features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definisci il numero di feature da selezionare
k = 68

# Applicare SelectKBest con ANOVA
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X_scaled, y)

# Ottenere gli indici delle feature selezionate
selected_features_indices = selector.get_support(indices=True)

# Ottenere i punteggi delle features
feature_scores = selector.scores_

# Ottenere i nomi delle colonne selezionate
selected_feature_names = X.columns[selected_features_indices].tolist()

# Creare un DataFrame per visualizzare i punteggi
scores_df = pd.DataFrame({
    'Feature': X.columns,
    'Score': feature_scores
})

# Ordinare i punteggi in ordine decrescente
scores_df = scores_df.sort_values(by='Score', ascending=False)

# Visualizzare i punteggi e i nomi delle features selezionate
print("Punteggi delle Features:")
print(scores_df)

print("\nLe 68 features migliori selezionate dall'ANOVA sono:")
print(selected_feature_names)



Colonne costanti:
['frame.frame.encap_type', 'frame.frame.offset_shift', 'frame.frame.marked', 'frame.frame.ignored', 'eth.eth.dst_raw[1]', 'eth.eth.dst_raw[2]', 'eth.eth.dst_raw[3]', 'eth.eth.dst_raw[4]', 'eth.eth.dst_tree.eth.dst_resolved_raw[1]', 'eth.eth.dst_tree.eth.dst_resolved_raw[2]', 'eth.eth.dst_tree.eth.dst_resolved_raw[3]', 'eth.eth.dst_tree.eth.dst_resolved_raw[4]', 'eth.eth.dst_tree.eth.dst.oui_raw[1]', 'eth.eth.dst_tree.eth.dst.oui_raw[2]', 'eth.eth.dst_tree.eth.dst.oui_raw[3]', 'eth.eth.dst_tree.eth.dst.oui_raw[4]', 'eth.eth.dst_tree.eth.addr_raw[1]', 'eth.eth.dst_tree.eth.addr_raw[2]', 'eth.eth.dst_tree.eth.addr_raw[3]', 'eth.eth.dst_tree.eth.addr_raw[4]', 'eth.eth.dst_tree.eth.addr_resolved_raw[1]', 'eth.eth.dst_tree.eth.addr_resolved_raw[2]', 'eth.eth.dst_tree.eth.addr_resolved_raw[3]', 'eth.eth.dst_tree.eth.addr_resolved_raw[4]', 'eth.eth.dst_tree.eth.addr.oui_raw[1]', 'eth.eth.dst_tree.eth.addr.oui_raw[2]', 'eth.eth.dst_tree.eth.addr.oui_raw[3]', 'eth.eth.dst_tree.

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:776: UserWarning: k=68 is greater than n_features=37. All the features will be returned.
  warnings.warn(


In [18]:
listel=selected_feature_names
print(listel)
print(len(listel))

['frame.frame.time_epoch', 'frame.frame.time_delta', 'frame.frame.time_delta_displayed', 'frame.frame.time_relative', 'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len', 'eth.eth.dst_raw[0]', 'eth.eth.dst', 'eth.eth.dst_tree.eth.dst_resolved_raw[0]', 'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui', 'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr', 'eth.eth.dst_tree.eth.addr_resolved_raw[0]', 'eth.eth.dst_tree.eth.addr.oui_raw[0]', 'eth.eth.dst_tree.eth.addr.oui', 'eth.eth.dst_tree.eth.dst.lg_raw[0]', 'eth.eth.dst_tree.eth.dst.lg', 'eth.eth.dst_tree.eth.lg_raw[0]', 'eth.eth.dst_tree.eth.lg', 'eth.eth.dst_tree.eth.dst.ig_raw[0]', 'eth.eth.dst_tree.eth.dst.ig', 'eth.eth.dst_tree.eth.ig_raw[0]', 'eth.eth.dst_tree.eth.ig', 'eth.eth.src_raw[0]', 'eth.eth.src', 'eth.eth.src_tree.eth.src_resolved_raw[0]', 'eth.eth.src_tree.eth.src.oui_raw[0]', 'eth.eth.src_tree.eth.src.oui', 'eth.eth.src_tree.eth.src.oui_resolved_raw[0]', 'eth.eth.src_tree.eth.a

In [19]:
df_clean=df_clean[listel]
df_pf_1m_05h=df_pf_1m_05h[listel]
df_pf_5m_05h=df_pf_5m_05h[listel]
df_pf_15m_05h=df_pf_15m_05h[listel]

In [20]:
fn.df_convert2(df_clean,listel)
fn.df_convert2(df_pf_1m_05h,listel)



,frame.frame.time_epoch,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,frame.frame.cap_len,eth.eth.dst_raw[0],eth.eth.dst,eth.eth.dst_tree.eth.dst_resolved_raw[0],...,eth.eth.src_tree.eth.src_resolved_raw[0],eth.eth.src_tree.eth.src.oui_raw[0],eth.eth.src_tree.eth.src.oui,eth.eth.src_tree.eth.src.oui_resolved_raw[0],eth.eth.src_tree.eth.addr_raw[0],eth.eth.src_tree.eth.addr,eth.eth.src_tree.eth.addr_resolved_raw[0],eth.eth.src_tree.eth.addr.oui_raw[0],eth.eth.src_tree.eth.addr.oui,eth.eth.src_tree.eth.addr.oui_resolved_raw[0]
0,1.533746e+09,0.000000,0.000000,0.000000,1,60,60,553850065211,553850065211,553850065211,...,79556642095225,4741945,4741945,79556642095225,79556642095225,79556642095225,79556642095225,4741945,4741945,79556642095225
1,1.533746e+09,0.010306,0.010306,0.010306,2,60,60,79556642095225,79556642095225,79556642095225,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211
2,1.533746e+09,0.001227,0.001227,0.011533,3,60,60,79556642095225,79556642095225,79556642095225,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211
3,1.533746e+09,0.000023,0.000023,0.011556,4,60,60,553850065211,553850065211,553850065211,...,79556642095225,4741945,4741945,79556642095225,79556642095225,79556642095225,79556642095225,4741945,4741945,79556642095225
4,1.533746e+09,0.000202,0.000202,0.011758,5,60,60,79556642095225,79556642095225,79556642095225,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47366,1.533748e+09,0.002414,0.002414,1798.877564,47367,85,85,52237803166,52237803166,52237803166,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211
47367,1.533748e+09,0.215134,0.215134,1799.092698,47368,60,60,553850065211,553850065211,553850065211,...,52237803166,3113,3113,52237803166,52237803166,52237803166,52237803166,3113,3113,52237803166
47368,1.533748e+09,0.094448,0.094448,1799.187146,47369,66,66,553850065211,553850065211,553850065211,...,52237803166,3113,3113,52237803166,52237803166,52237803166,52237803166,3113,3113,52237803166
47369,1.533748e+09,0.009739,0.009739,1799.196885,47370,85,85,52237803166,52237803166,52237803166,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211


In [21]:
fn.df_convert2(df_pf_5m_05h,listel)
fn.df_convert2(df_pf_15m_05h,listel)

,frame.frame.time_epoch,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,frame.frame.cap_len,eth.eth.dst_raw[0],eth.eth.dst,eth.eth.dst_tree.eth.dst_resolved_raw[0],...,eth.eth.src_tree.eth.src_resolved_raw[0],eth.eth.src_tree.eth.src.oui_raw[0],eth.eth.src_tree.eth.src.oui,eth.eth.src_tree.eth.src.oui_resolved_raw[0],eth.eth.src_tree.eth.addr_raw[0],eth.eth.src_tree.eth.addr,eth.eth.src_tree.eth.addr_resolved_raw[0],eth.eth.src_tree.eth.addr.oui_raw[0],eth.eth.src_tree.eth.addr.oui,eth.eth.src_tree.eth.addr.oui_resolved_raw[0]
0,1.526899e+09,0.000000,0.000000,0.000000,1,60,60,553850065211,553850065211,553850065211,...,79556642095225,4741945,4741945,79556642095225,79556642095225,79556642095225,79556642095225,4741945,4741945,79556642095225
1,1.526899e+09,0.009869,0.009869,0.009869,2,60,60,79556642095225,79556642095225,79556642095225,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211
2,1.526899e+09,0.000843,0.000843,0.010712,3,60,60,79556642095225,79556642095225,79556642095225,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211
3,1.526899e+09,0.000019,0.000019,0.010731,4,60,60,553850065211,553850065211,553850065211,...,79556642095225,4741945,4741945,79556642095225,79556642095225,79556642095225,79556642095225,4741945,4741945,79556642095225
4,1.526899e+09,0.000178,0.000178,0.010909,5,60,60,79556642095225,79556642095225,79556642095225,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29687,1.526900e+09,0.009229,0.009229,1798.146323,29688,60,60,79556642095225,79556642095225,79556642095225,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211
29688,1.526900e+09,0.008271,0.008271,1798.154594,29689,60,60,79556642095225,79556642095225,79556642095225,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211
29689,1.526900e+09,0.000830,0.000830,1798.155424,29690,66,66,79556642095225,79556642095225,79556642095225,...,553850065211,33012,33012,553850065211,553850065211,553850065211,553850065211,33012,33012,553850065211
29690,1.526900e+09,0.058277,0.058277,1798.213701,29691,64,64,1652522221568,1652522221568,1652522221568,...,229240264868019,13663784,13663784,229240264868019,229240264868019,229240264868019,229240264868019,13663784,13663784,229240264868019


In [22]:

# Supponiamo che df_clean e df_pf_1m_05h siano i tuoi DataFrame

# Converti gli interi in pd.Timedelta, se necessario

df_clean['frame.frame.time_epoch'] = pd.to_timedelta(df_clean['frame.frame.time_epoch'], unit='s')
df_pf_1m_05h['frame.frame.time_epoch'] = pd.to_timedelta(df_pf_1m_05h['frame.frame.time_epoch'], unit='s')


In [23]:
df_pf_5m_05h['frame.frame.time_epoch'] = pd.to_timedelta(df_pf_5m_05h['frame.frame.time_epoch'], unit='s')
df_pf_15m_05h['frame.frame.time_epoch'] = pd.to_timedelta(df_pf_15m_05h['frame.frame.time_epoch'], unit='s')

In [24]:
# Calcola i cambiamenti di tempo
df_clean['capture_id'] = (df_clean['frame.frame.time_epoch'].diff() > pd.Timedelta(seconds=300)).cumsum()
df_pf_1m_05h['capture_id'] = (df_pf_1m_05h['frame.frame.time_epoch'].diff() > pd.Timedelta(seconds=300)).cumsum()
df_pf_5m_05h['capture_id'] = (df_pf_5m_05h['frame.frame.time_epoch'].diff() > pd.Timedelta(seconds=300)).cumsum()
df_pf_15m_05h['capture_id'] = (df_pf_15m_05h['frame.frame.time_epoch'].diff() > pd.Timedelta(seconds=300)).cumsum()

In [25]:
df_clean_shape = df_clean['capture_id'].nunique()
df_pf_1m_05h_shape = df_pf_1m_05h['capture_id'].nunique()
df_pf_5m_05h_shape = df_pf_5m_05h['capture_id'].nunique()
df_pf_15m_05h_shape = df_pf_15m_05h['capture_id'].nunique()
print(df_clean_shape,df_pf_1m_05h_shape,df_pf_5m_05h_shape, df_pf_15m_05h_shape)

1 1 1 1


In [26]:
df_clean['label'] = 0
df_pf_1m_05h['label'] = 1
df_pf_5m_05h['label'] = 1
df_pf_15m_05h['label'] = 1

In [27]:
concat = pd.concat([df_clean, df_pf_1m_05h, df_pf_5m_05h, df_pf_15m_05h], axis=0)

In [28]:
print(concat.columns.tolist())

['frame.frame.time_epoch', 'frame.frame.time_delta', 'frame.frame.time_delta_displayed', 'frame.frame.time_relative', 'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len', 'eth.eth.dst_raw[0]', 'eth.eth.dst', 'eth.eth.dst_tree.eth.dst_resolved_raw[0]', 'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui', 'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr', 'eth.eth.dst_tree.eth.addr_resolved_raw[0]', 'eth.eth.dst_tree.eth.addr.oui_raw[0]', 'eth.eth.dst_tree.eth.addr.oui', 'eth.eth.dst_tree.eth.dst.lg_raw[0]', 'eth.eth.dst_tree.eth.dst.lg', 'eth.eth.dst_tree.eth.lg_raw[0]', 'eth.eth.dst_tree.eth.lg', 'eth.eth.dst_tree.eth.dst.ig_raw[0]', 'eth.eth.dst_tree.eth.dst.ig', 'eth.eth.dst_tree.eth.ig_raw[0]', 'eth.eth.dst_tree.eth.ig', 'eth.eth.src_raw[0]', 'eth.eth.src', 'eth.eth.src_tree.eth.src_resolved_raw[0]', 'eth.eth.src_tree.eth.src.oui_raw[0]', 'eth.eth.src_tree.eth.src.oui', 'eth.eth.src_tree.eth.src.oui_resolved_raw[0]', 'eth.eth.src_tree.eth.a

In [29]:
concat.columns

Index(['frame.frame.time_epoch', 'frame.frame.time_delta',
       'frame.frame.time_delta_displayed', 'frame.frame.time_relative',
       'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len',
       'eth.eth.dst_raw[0]', 'eth.eth.dst',
       'eth.eth.dst_tree.eth.dst_resolved_raw[0]',
       'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui',
       'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr',
       'eth.eth.dst_tree.eth.addr_resolved_raw[0]',
       'eth.eth.dst_tree.eth.addr.oui_raw[0]', 'eth.eth.dst_tree.eth.addr.oui',
       'eth.eth.dst_tree.eth.dst.lg_raw[0]', 'eth.eth.dst_tree.eth.dst.lg',
       'eth.eth.dst_tree.eth.lg_raw[0]', 'eth.eth.dst_tree.eth.lg',
       'eth.eth.dst_tree.eth.dst.ig_raw[0]', 'eth.eth.dst_tree.eth.dst.ig',
       'eth.eth.dst_tree.eth.ig_raw[0]', 'eth.eth.dst_tree.eth.ig',
       'eth.eth.src_raw[0]', 'eth.eth.src',
       'eth.eth.src_tree.eth.src_resolved_raw[0]',
       'eth.eth.src_tree.eth.src.oui_r

In [30]:
concat.dtypes

frame.frame.time_epoch                           timedelta64[ns]
frame.frame.time_delta                                   float64
frame.frame.time_delta_displayed                         float64
frame.frame.time_relative                                float64
frame.frame.number                                         int32
frame.frame.len                                            int32
frame.frame.cap_len                                        int32
eth.eth.dst_raw[0]                                         int64
eth.eth.dst                                                int64
eth.eth.dst_tree.eth.dst_resolved_raw[0]                   int64
eth.eth.dst_tree.eth.dst.oui_raw[0]                        int64
eth.eth.dst_tree.eth.dst.oui                               int32
eth.eth.dst_tree.eth.addr_raw[0]                           int64
eth.eth.dst_tree.eth.addr                                  int64
eth.eth.dst_tree.eth.addr_resolved_raw[0]                  int64
eth.eth.dst_tree.eth.addr

In [31]:
concat.drop(columns=['frame.frame.time_epoch'], inplace=True)

In [32]:
print(concat)

       frame.frame.time_delta  frame.frame.time_delta_displayed  \
0                    0.000000                          0.000000   
1                    0.055757                          0.055757   
2                    0.094665                          0.094665   
3                    0.003102                          0.003102   
4                    0.214236                          0.214236   
...                       ...                               ...   
29687                0.009229                          0.009229   
29688                0.008271                          0.008271   
29689                0.000830                          0.000830   
29690                0.058277                          0.058277   
29691                0.143918                          0.143918   

       frame.frame.time_relative  frame.frame.number  frame.frame.len  \
0                       0.000000                   1               64   
1                       0.055757                 

In [33]:
print(concat)

       frame.frame.time_delta  frame.frame.time_delta_displayed  \
0                    0.000000                          0.000000   
1                    0.055757                          0.055757   
2                    0.094665                          0.094665   
3                    0.003102                          0.003102   
4                    0.214236                          0.214236   
...                       ...                               ...   
29687                0.009229                          0.009229   
29688                0.008271                          0.008271   
29689                0.000830                          0.000830   
29690                0.058277                          0.058277   
29691                0.143918                          0.143918   

       frame.frame.time_relative  frame.frame.number  frame.frame.len  \
0                       0.000000                   1               64   
1                       0.055757                 

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Supponendo che il DataFrame 'concat' contenga già le catture (con capture_id) e la colonna 'label'
# In questo caso, le caratteristiche sono già aggregate per cattura

# Separare le caratteristiche (escludendo 'label') e l'etichetta 'label'
X = concat.drop(['label', 'capture_id'], axis=1)  # Rimuovi anche 'capture_id' perché non è una caratteristica
y = concat['label']  # Etichette per la classificazione

# Verifica delle dimensioni delle caratteristiche e delle etichette
print("Dimensioni di X:", X.shape)
print("Dimensioni di y:", y.shape)

# Gestione dei valori NaN: sostituiamo i valori NaN con 0 (o puoi usare un altro metodo come la media)
X = X.fillna(0)

# Suddividere i dati in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# Normalizzazione delle caratteristiche
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Creare e addestrare il modello k-NN
knn_model = KNeighborsClassifier(n_neighbors=5)  # Puoi regolare il valore di k
knn_model.fit(X_train, y_train)

# Fare previsioni
y_pred = knn_model.predict(X_test)

# Stampa dei risultati
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Dimensioni di X: (196385, 36)
Dimensioni di y: (196385,)


Confusion Matrix:
[[20582  1063]
 [  517 36754]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.95      0.96     21645
           1       0.97      0.99      0.98     37271

    accuracy                           0.97     58916
   macro avg       0.97      0.97      0.97     58916
weighted avg       0.97      0.97      0.97     58916

